In [1]:
from datetime import datetime, timedelta
import time
from netdata_pandas.data import get_alarm_log
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

def print_itemsets(itemsets, mode='alarm', n_max=500):
    print(f'-'*50)
    print(f'{mode} itemsets')
    print(f'-'*50)
    i = 0
    if len(itemsets) > 0:
        for _, row in itemsets.iterrows():
            if i <= n_max:
                s = round(row.support*100,2)
                ii = set(row.itemsets)
                if mode == 'alarm':
                    print(f'{s}% contained the alarms: {ii}')
                elif mode == 'chart':
                    print(f'{s}% related to the charts: {ii}')
                i += 1
            else:
                continue
    else:
        print(None)


def print_rules(rules, mode='alarm', n_max=500):
    print(f'-'*50)
    print(f'{mode} association rules')
    print(f'-'*50)
    i = 0
    if len(rules) > 0:
        for _, row in rules.iterrows():
            if i <= n_max:
                c = round(row.confidence*100,2)
                s = round(row.support*100,2)
                l = round(row.lift,2)
                a_set = set(row.antecedents)
                c_set = set(row.consequents)
                print(f'{c}% probability that {a_set} -> {c_set} (co-occurrence={s}%, lift={l})')
                i += 1
            else:
                continue
    else:
        print(None)

In [2]:
# inputs
host = 'london.my-netdata.io'
window = '1m'
min_support = 0.1   
min_threshold = 0.1
since_hours_ago = None # None or int
last_n_alarms = 100 # None or int
max_n = None

In [3]:
# get alarm log data
df = get_alarm_log(host)
df = df[df['status'].isin(['WARNING', 'CRITICAL'])]
df = df.sort_values('when')
print(df.shape)

(422, 33)


In [4]:
if since_hours_ago:
    df = df[df['when'] >= (datetime.now()-timedelta(hours=since_hours_ago))]

if last_n_alarms:
    df = df.tail(last_n_alarms).copy()

print(df.shape)

(100, 33)


In [5]:
# make transactions dataset
td = pd.Timedelta(window)
f_alarm = lambda x, y: df.loc[df['when'].between(y - td, y + td), 'name'].tolist()
df['alarm_basket'] = [f_alarm(k, v) for k, v in df['when'].items()]
f_chart = lambda x, y: df.loc[df['when'].between(y - td, y + td), 'chart'].tolist()
df['chart_basket'] = [f_chart(k, v) for k, v in df['when'].items()]

if max_n:
    if len(df) > max_n:
        df = df.sample(max_n)
        print(df.shape)

alarm_dataset = df['alarm_basket'].values.tolist()
chart_dataset = df['chart_basket'].values.tolist()

In [6]:
# get some varialbes to print later
n_alarms_analyzed = len(alarm_dataset)
when_min = df['when'].min()
when_max = df['when'].max()

In [7]:
# alarm
te_alarm = TransactionEncoder()
te_ary_alarm = te_alarm.fit(alarm_dataset).transform(alarm_dataset)
df_alarm_tx = pd.DataFrame(te_ary_alarm, columns=te_alarm.columns_)
itemsets_alarm = fpgrowth(df_alarm_tx, min_support=min_support, use_colnames=True).sort_values('support', ascending=False)
rules_alarm = association_rules(itemsets_alarm, metric="confidence", min_threshold=min_threshold).sort_values('support', ascending=False)

# chart
te_chart = TransactionEncoder()
te_ary_chart = te_chart.fit(chart_dataset).transform(chart_dataset)
df_chart_tx = pd.DataFrame(te_ary_chart, columns=te_chart.columns_)
itemsets_chart = fpgrowth(df_chart_tx, min_support=min_support, use_colnames=True).sort_values('support', ascending=False)
rules_chart = association_rules(itemsets_chart, metric="confidence", min_threshold=min_threshold).sort_values('support', ascending=False)

In [8]:
# print results
print(f'\n{n_alarms_analyzed} alarms analyzed (spanning {when_min} to {when_max})')

# alarm
print_itemsets(itemsets_alarm, 'alarm', 50)
print_rules(rules_alarm, 'alarm', 50)

# chart
print_itemsets(itemsets_chart, 'chart', 50)
print_rules(rules_chart, 'chart', 50)


100 alarms analyzed (spanning 2020-12-12 00:49:27 to 2020-12-18 17:35:55)
--------------------------------------------------
alarm itemsets
--------------------------------------------------
96.0% contained the alarms: {'1m_successful'}
55.0% contained the alarms: {'1m_bad_requests'}
55.0% contained the alarms: {'1m_successful', '1m_bad_requests'}
--------------------------------------------------
alarm association rules
--------------------------------------------------
57.29% probability that {'1m_successful'} -> {'1m_bad_requests'} (co-occurrence=55.0%, lift=1.04)
100.0% probability that {'1m_bad_requests'} -> {'1m_successful'} (co-occurrence=55.0%, lift=1.04)
--------------------------------------------------
chart itemsets
--------------------------------------------------
96.0% related to the charts: {'web_log_nginx.response_statuses'}
--------------------------------------------------
chart association rules
--------------------------------------------------
None


In [9]:
rules_alarm

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1m_successful),(1m_bad_requests),0.96,0.55,0.55,0.572917,1.041667,0.022,1.053659
1,(1m_bad_requests),(1m_successful),0.55,0.96,0.55,1.000000,1.041667,0.022,inf
